# Convergence Training

In convergence training, we implemented a method to continiously generate new data until a reference classifier can't get fooled. In our experiments we accomplished to generate 60.000 new data entries for mnist, while maintaining 95 % prediction accuracy.

In [ ]:
# General
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Configuration
from omegaconf import OmegaConf

# Import src folder
import sys
import os
from pathlib import Path

home = Path(os.path.abspath('')).parent
sys.path.append(os.path.join(home, "src"))

# Now import alva
from alva import generate_samples_with_iterative_epsilons

# Import custom modules
from models import LeNet5, CGanGenerator
from data import mnist, PerturbatedMnist
from utils import set_random_seed, split_tensor_random, unnormalize_tensor
from training import training_loop

### Load config and Hyperparameters

In [ ]:
# Paths
CFG_PATH_TRAIN=r"config\training_config.yaml"
CFG_PATH_DATA = r"config\mnist_data_config.yaml"
CFG_PATH_GENERATOR_HPARAMS = r"config\generator_hparams_config.yaml"
CFG_PATH_CONVERGENCE = r"config\convergence_config.yaml"

# Load Config
cfg_hyperparams = OmegaConf.load(CFG_PATH_TRAIN)
cfg_data = OmegaConf.load(CFG_PATH_DATA)
cfg_generator = OmegaConf.load(CFG_PATH_GENERATOR_HPARAMS)
cfg_convergence = OmegaConf.load(CFG_PATH_CONVERGENCE)

# Set random seed
set_random_seed(0)

# Get the device for cuda optimization
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set Hyperparameters - Training
BATCH_SIZE = cfg_hyperparams.BATCH_SIZE
LEARNING_RATE = cfg_hyperparams.LEARNING_RATE
N_EPOCHS = cfg_hyperparams.N_EPOCHS
# Set Hyperparameters - Convergence 
N_EPSILON_EPOCHS = cfg_convergence.N_CONV_EPOCHS
N_GENERATED_SAMPLES = cfg_convergence.N_GENERATED_SAMPLES
N_TIMEOUT_TRIES = cfg_convergence.N_TIMEOUT
CLASSES = cfg_data.CLASSES

# Load Data
root = os.path.join(os.path.abspath(os.path.curdir),cfg_data.ROOT)
per_training_data = PerturbatedMnist(root,'training', transform = mnist.get_standard_transformation())
per_test_data = PerturbatedMnist(root, 'test', transform = mnist.get_standard_transformation())

#Load the Generator
generator = CGanGenerator(cfg_generator.LATENT_DIM, tuple(cfg_generator.OUTPUT_DIM)).to(DEVICE)
generator.load_state_dict(torch.load(r"models\state_dicts\conditional_gan_generator.pt"))

Helper method for convergence training  (see [alva example](alva_example.ipynb) for more details)

In [ ]:
def generate_samples(classifier, generator, device, target, n_generated_samples, n_timeout, print_info : bool = True):
    """
    Generates samples 
    """
    # Generate samples
    (z, y, per_z, per_y) = generate_samples_with_iterative_epsilons(classifier, generator, device, target, n_generated_samples, n_timeout)
    
    if print_info:
        print(f"Generated {len(z)} adversarial samples with generator")
    
    x, per_x = generator(z).detach().cpu(), generator(per_z).detach().cpu()

    return x, y, per_x, per_y

### Convergence Training consists of three steps

Convergence Training consists of following steps:

1. Train a classifier $C$ on a dataset $D$.
2. Use ALVA to generate $n$ samples from each class in $D$.
3. Add the generated samples to $D$.
4. Repeat steps 1-3 until ALVA can no longer "fool" $C$.

The steps are also marked in the code below.

In [ ]:
for epsilon_epoch in range(N_EPSILON_EPOCHS):
    # Declare experiment specific variables
    run_name = f"{epsilon_epoch:03}"

    # Reload data and create Dataloader
    per_training_data.load_data()
    per_test_data.load_data()
    train_loader = DataLoader(per_training_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(per_test_data, batch_size=BATCH_SIZE, shuffle=True)

    # Start run
    print(f'\n--------------------------')
    print(f'Executing Experiment: #{run_name}')
    print(f'\nPerturbated images: {round(per_training_data.get_perturbated_percentage(), 3)}%')
    print(f'--------------------------\n')

    # Reinitialize model
    classifier = LeNet5().to(DEVICE)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    # Train the model on the dataset - STEP 1
    print("\nTRAINING\n")
    classifier, optimizer, metrics = training_loop(classifier, criterion, optimizer, train_loader, test_loader, N_EPOCHS, DEVICE)

    # Execute the pipeline to generate samples for every class
    per_xs = []
    targets = []
    all_target_figures = []
    epsilons  = []

    # STEP 2
    print("\nGENERATING\n")
    for target in CLASSES:
        print("\nGenerating data with class  " + str(target) + "\n")

        # Generating images and storing figures 
        x, y, per_x, per_y = generate_samples(classifier, generator, DEVICE, target, N_GENERATED_SAMPLES, BATCH_SIZE, print_info=False)

        # Append data
        per_xs.append(unnormalize_tensor(per_x))
        targets.append(torch.full((1, per_x.shape[0]), target, dtype=int))

    # Concatenate date and split
    per_xs = torch.cat(per_xs).view(-1, 28,28)
    targets = torch.cat(targets, dim=1).view(-1)
    # Process data
    x_test, y_test, x_train, y_train = split_tensor_random(per_xs, targets)

    # Save generated data - STEP 3
    train_path = os.path.join(per_training_data.get_perturbated_data_dir() , f'{run_name}_training.pt')
    test_path = os.path.join(per_test_data.get_perturbated_data_dir(), f'{run_name}_test.pt')
    torch.save((x_train, y_train), train_path)
    torch.save((x_test, y_test), test_path)